In [1]:
from fastai.vision.all import *
from fastbook import *

In [2]:
torch.cuda.is_available()

True

## Image loading

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
def loadImages(paths):
    images = [Image.open(p) for p in paths]
    classes = [p.parent.name for p in paths]
    return (images, classes)
    
loadImages([
    path/"training"/"5"/"0.png",
    path/"training"/"9"/"10003.png"
])

([<PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>],
 ['5', '9'])

In [5]:
import glob

def allImages(path):
    return list(path.glob("**/*.png"))

testImages = allImages(path)
assert len(testImages) > 0
for image in testImages: assert image.name.endswith(".png") 
len(testImages), type(testImages[0])

(70000, pathlib.WindowsPath)

## Model

In [26]:
class Model:
    def __init__(self, imageSize, categoryCount):
        self.w1 = self.__initParams(imageSize[0] * imageSize[1], categoryCount)
        self.b1 = self.__initParams(categoryCount)
        self.params = [self.w1, self.b1]
        
    def applyModel(self, batch):
        return batch@self.w1 + self.b1
    
    def fit(self, lr):
        for p in self.params:
            #print("adopting model by " + str(p.grad))
            p.data -= p.grad.data * lr
            p.grad = None
    
    def __initParams(self, *size):
        return torch.rand(size).requires_grad_()

## Learning preparation

In [7]:
def mse(targetProbabilities, predictedProbabilities):
    return torch.square(targetProbabilities - predictedProbabilities).mean()

testTargets = tensor([1, 0, 1])
test1 = mse(testTargets, tensor([0.9, 0.2, 0.4]))
test2 = mse(testTargets, tensor([0.9, 0.2, 0.5]))
test1, test2

(tensor(0.1367), tensor(0.1000))

In [8]:
def sigmoid(x): return 1/(1+torch.exp(-x))

sigmoid(tensor([-4, -1, 1, 4]))

tensor([0.0180, 0.2689, 0.7311, 0.9820])

In [9]:
def mse_loss(targetProbabilities, predictions):
    return mse(targetProbabilities, sigmoid(predictions))
    
test1 = mse_loss(tensor([1, 0, 1]), tensor([10, -3, 0]))
test2 = mse_loss(tensor([1, 0, 1]), tensor([10, -3, 1]))
test1, test2

(tensor(0.0841), tensor(0.0249))

In [10]:
def calculateAccuracy(targetPredictions, predictions):
    border = 0.5
    return ((predictions > border) == (targetPredictions > border)).float().mean()

calculateAccuracy(tensor([1, 0, 1]), tensor([0.2, 0, 0.61]))

tensor(0.6667)

In [11]:
class BatchLoader:
    def __init__(self, images, batchSize):
        self.images = images.copy()
        random.shuffle(self.images)
        self.batchSize = batchSize
        self.nextBatch = 0 
    def nextBatch(self):
        batchStartIndex = min(self.batchSize * self.nextBatch, len(self.images))
        batchEndIndex = min(startBatchFrom + self.batchSize, len(self.images))
        if batchStartIndex == batchEndIndex:
            return None
        else:
            return self.images[batchStartIndex:batchEndIndex]

In [12]:
class BatchLoader:
    def __init__(self, items, batchSize):
        self.items = items.copy()
        random.shuffle(self.items)
        self.batchSize = batchSize
        self.nextBatch = 0 
    def getNextBatch(self):
        batchStartIndex = min(self.batchSize * self.nextBatch, len(self.items))
        batchEndIndex = min(batchStartIndex + self.batchSize, len(self.items))
        if batchStartIndex == batchEndIndex:
            return None
        else:
            self.nextBatch += 1
            return self.items[batchStartIndex:batchEndIndex]
       
    
testItems = [1, 2, 3, 4, 5, 6, 7]
testLoader = BatchLoader(testItems, 4)
receivedItems = testLoader.getNextBatch() + testLoader.getNextBatch()
receivedItems.sort()
assert testItems == receivedItems, "received aren't the same as tests " + receivedItems
assert testLoader.getNextBatch() == None

In [13]:
def loadTrainingBatch(paths):
    images = [tensor(Image.open(p)).view(-1) for p in paths]
    classes = [p.parent.name for p in paths]
    return (torch.stack(images).float()/255, classes)
    
testBatchLoader = BatchLoader(allImages(path/"training"), 100)
trainingImages, trainingClasses = loadTrainingBatch(testBatchLoader.getNextBatch())
assert len(trainingImages) == len(trainingClasses)

In [14]:
def numberToPrediction(number):
    return [1 if i == number else 0 for i in range(10)]

def targetPredictions(classes):
    return tensor([numberToPrediction(int(c)) for c in classes])

targetPredictions(["0", "5", "9"])

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

### The learning loop

In [33]:
def performEpoach(model, trainingSet, batchSize, lr):
    batchLoader = BatchLoader(trainingSet, batchSize)
    batch = batchLoader.getNextBatch()
    while batch != None:
        traingingImages, trainingClasses = loadTrainingBatch(batch)
        predictions = sigmoid(model.applyModel(traingingImages))
        target = targetPredictions(trainingClasses)
        loss = mse(target, predictions)
        loss.backward()
        print("gradient")
        print(model.w1.grad.mean())
        print("params")
        print(model.w1.mean())
        model.fit(lr)
        #accuracy = calculateAccuracy(target, predictions)
        #print("Target: " + str(target))
        #print("Predicted: " + str(predictions))
        print("Loss: " + str(loss.item()))
        #print("Accuracy: " + str(accuracy))
        batch = batchLoader.getNextBatch()
    
model = Model((28, 28), 10)    
performEpoach(model, allImages(path/"training"), 1000, 0.1)

gradient
tensor(3.6290e-11)
params
tensor(0.5005, grad_fn=<MeanBackward0>)
Loss: 0.8999999761581421
gradient
tensor(2.5411e-11)
params
tensor(0.5005, grad_fn=<MeanBackward0>)
Loss: 0.8999999761581421
gradient
tensor(8.5717e-11)
params
tensor(0.5005, grad_fn=<MeanBackward0>)
Loss: 0.8999999761581421
gradient
tensor(2.7580e-10)
params
tensor(0.5005, grad_fn=<MeanBackward0>)
Loss: 0.8999999165534973
gradient
tensor(4.2144e-11)
params
tensor(0.5005, grad_fn=<MeanBackward0>)
Loss: 0.8999999761581421
gradient
tensor(7.3416e-11)
params
tensor(0.5005, grad_fn=<MeanBackward0>)
Loss: 0.8999999761581421
gradient
tensor(1.8346e-11)
params
tensor(0.5005, grad_fn=<MeanBackward0>)
Loss: 0.8999999761581421
gradient
tensor(3.3813e-11)
params
tensor(0.5005, grad_fn=<MeanBackward0>)
Loss: 0.8999999761581421
gradient
tensor(1.0172e-10)
params
tensor(0.5005, grad_fn=<MeanBackward0>)
Loss: 0.8999999761581421
gradient
tensor(7.6641e-11)
params
tensor(0.5005, grad_fn=<MeanBackward0>)
Loss: 0.8999999761581421
